For scraping data from the Global Footprint Network.
https://www.footprintnetwork.org/

Countrycode Canada: 33

data types: 
biocapacity per capita (gha / person): 'BCpc'
biocapacity (gha): 'BCtot'
ecological footprint per capita (gha / person): 'EFCpc'
ecological footprint (gha): 'EFCtot'
number of earths: 'earth'


In [1]:
import pandas as pd
import requests
import json

In [2]:
def download(country, datastring): 
    
    countrycode = {'Canada':33,
                    'World':5001}[country]
    headers = {
        'Accept': 'application/json',
    }
    key = '13CMLmdbBei8dBCKf4tv3Gq7u2Vkg89so04K2reij4tm0l8o17r6'
    url_base = 'https://api.footprintnetwork.org/v1/data/'

    yearout = []
    valueout = []
    years = list(range(1961,2017))

    for year in years:
        url = url_base+str(countrycode)+'/'+str(year)+'/'+datastring
        r = requests.get(url, headers=headers, auth=('any-user-name', key))
        yearout.append((json.loads(r.content))[0]['year'])
        valueout.append((json.loads(r.content))[0]['value'])

    df = pd.DataFrame({'year':yearout, datastring+'_'+country:valueout})
    return df 


In [3]:
def mergedata(country):
  
    df = pd.DataFrame({'year':list(range(1961,2017))})
    
    for datastring in ['BCpc','BCtot','EFCpc','EFCtot','earth']:
        dg = download(country,datastring)
        df = df.merge(dg, on='year', how='left')
        
    return df

In [4]:
def mergecountries():
    
    df = pd.DataFrame({'year':list(range(1961,2017))})
    
    for country in ['Canada', 'World']:
        dg = mergedata(country)
        df = df.merge(dg, on = 'year', how='left')
        
    return df


In [5]:
df = mergecountries()

In [9]:
import datetime as dt
leaps = [1964,1968,1972,1976,1980,1984,1988,1992,1996,2000,2004,2008,2012,2016] 

for i in df['year']:
    if i in leaps:
        df['od_world'] = (df['BCtot_World'] / df['EFCtot_World']) * 366
        df['od_can_pc'] = (df['BCpc_World']/ df['EFCpc_Canada']) * 366
    else:
        df['od_world'] = (df['BCtot_World'] / df['EFCtot_World']) * 365
        df['od_can_pc'] = (df['BCpc_World']/ df['EFCpc_Canada']) * 365
    

df['odate_world'] = [pd.to_datetime('1/1/1900') + dt.timedelta(days=i) for i in df['od_world']]
df['odate_can'] = [pd.to_datetime('1/1/1900') + dt.timedelta(days=i) for i in df['od_can_pc']]



In [26]:
df.to_csv('~/code/footprint/static/footprintnetwork.csv')
#df2=pd.read_csv('~/code/footprint/static/footprintnetwork.csv')